In [1]:
import KeypressModule as kp
from djitellopy import tello
import cv2
import time
import threading
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
kp.init()
me = tello.Tello()
me.connect()
print(me.get_battery())
me.streamon()

[INFO] tello.py - 107 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 422 - Send command: 'command'
[INFO] tello.py - 446 - Response command: 'ok'
[INFO] tello.py - 422 - Send command: 'streamon'
[INFO] tello.py - 446 - Response streamon: 'ok'


55


In [3]:
global img

def get_keyboard_input():

    if kp.get_key('q'):
        me.land()
        time.sleep(3)
    elif kp.get_key('e'):
        me.takeoff()

    elif kp.get_key('z'):
        cv2.imwrite(f'Resources/Images/{time.time()}.jpg', img)
        time.sleep(0.3)

In [4]:
hsvVals = [0, 0, 218, 179, 21, 255]
sensors = 3
threshold = 0.2
width, height = 480, 360

sensitivity = 3  # Less sensitive if number is high

weights = [-50, -25, 0, 25, 50]

fSpeed = 15
curve = 0


def thresholding(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsvVals[0], hsvVals[1], hsvVals[2]])
    upper = np.array([hsvVals[3], hsvVals[4], hsvVals[5]])
    mask = cv2.inRange(hsv, lower, upper)
    return mask


def getContours(imgThres, img):
    contours, hierarchy = cv2.findContours(imgThres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours) != 0:
        biggest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(biggest)
        cx = x + w // 2
        cy = y + h // 2

        cv2.drawContours(img, biggest, -1, (255, 0, 255), 7)
        cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)
    else:
        cx = width // 2

    return cx


def getSensorOutput(imgThres, sensors):
    imgs = np.hsplit(imgThres, sensors)
    totalPixels = (img.shape[1] // sensors) * img.shape[0]
    senOut = []
    for x, im in enumerate(imgs):
        pixelCount = cv2.countNonZero(im)
        if pixelCount > threshold * totalPixels:
            senOut.append(1)
        else:
            senOut.append(0)
        cv2.imshow(str(x), im)
    print(senOut)
    return senOut


def sendCommands(senOut, cx):
    global curve
    # Translation
    lr = (cx - width // 2) // sensitivity
    lr = int(np.clip(lr, -10, 10))
    if 2 > lr > -2:
        lr = 0
    # me.send_rc_control(lr, 0, 0, 0)

    # Rotation
    if senOut == [1, 0, 0]:
        curve = weights[0]
    elif senOut == [1, 1, 0]:
        curve = weights[1]
    elif senOut == [0, 1, 0]:
        curve = weights[2]
    elif senOut == [0, 1, 1]:
        curve = weights[3]
    elif senOut == [0, 0, 1]:
        curve = weights[4]
    elif senOut == [0, 0, 0]:
        curve = weights[2]
    elif senOut == [1, 1, 1]:
        curve = weights[2]
    elif senOut == [1, 0, 1]:
        curve = weights[2]

    me.send_rc_control(lr, fSpeed, 0, curve)

In [ ]:
while True:
    get_keyboard_input()
    img = me.get_frame_read().frame
    img = cv2.resize(img, (width, height))
    img = np.vsplit(img, sensors)[-1]

    imgThres = thresholding(img)
    cx = getContours(imgThres, img)  # For translation
    senOut = getSensorOutput(imgThres, sensors)  # For rotation
    sendCommands(senOut, cx)

    cv2.imshow('Output', img)
    cv2.imshow('Path', imgThres)
    cv2.waitKey(1)

[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] decode_slice_header error
[h264 @ 0x7f8e620eda00] no frame!
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] decode_slice_header error
[h264 @ 0x7f8e620eda00] no frame!
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] decode_slice_header error
[h264 @ 0x7f8e620eda00] no frame!
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] decode_slice_header error
[h264 @ 0x7f8e620eda00] no frame!
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] non-existing PPS 0 referenced
[h264 @ 0x7f8e620eda00] decode_slice_header error
[h264 @ 0x7f8e620eda00] no frame!
[h264 @ 0x7f8e620eda00] non-existing PPS

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'


[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc 0 15 0 0'


...

[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'


[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'


[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
[INFO] tello.py - 455 - Send command (no response expected): 'rc -10 15 0 0'
